In [1]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1


def get_game_info(thisgame_df, row):
    
    thisgame_df['visteam'] = row['info']['visteam']
    thisgame_df['hometeam'] = row['info']['hometeam']
    thisgame_df['date'] = pandas.to_datetime(row['info']['date'], errors='coerce')
    thisgame_df['time'] = row['info']['starttime']
    thisgame_df['daynight'] = row['info']['daynight']
    thisgame_df['site'] = row['info']['site']
    thisgame_df['length'] = pandas.to_timedelta(str(row['info']['timeofgame'])+' minutes', errors='ignore')
     
    return thisgame_df

def parse_events(thisgame_df):
#    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
#    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['play'] = thisgame_df['ev'].str.split('/').str.get(0)
#    thisgame_df['modifiers'] = thisgame_df['ev'].str.split('/',n=1).str.get(1)
#    thisgame_df['modifiers'] = thisgame_df['modifiers'].fillna('')
#    thisgame_df['baserunning'] = thisgame_df['modifiers'].str.split('.',n=1).str.get(1)
#    thisgame_df['baserunning'] = thisgame_df['baserunning'].fillna('')
#    thisgame_df['modifiers'] = thisgame_df['modifiers'].str.split('.').str.get(0)
    thisgame_df['mdfs'] = thisgame_df['ev'].str.split('/',n=1).str.get(1)
    thisgame_df['mdfs'] = thisgame_df['mdfs'].fillna('')
    thisgame_df['brun'] = thisgame_df['mdfs'].str.split('.',n=1).str.get(1)
    thisgame_df['brun'] = thisgame_df['brun'].fillna('')
    thisgame_df['mdfs'] = thisgame_df['mdfs'].str.split('.').str.get(0)
    
    return thisgame_df

In [2]:
# runs in about 5 minutes
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

ok


In [ ]:
#allgames_df.sample(10).index
#'MIN201204230', 'SEA201207290', 'LAN201304280', 'OAK201309010',
#       'SFN201409270', 'DET201306090', 'TEX201208240', 'ANA201404280',
#       'WAS201304260', 'CIN201609060'],


In [3]:
allgames_df = allgames_df.iloc[allgames_df.index == 'MIN201204230']
#allgames_df = allgames_df.sample(1)

#column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
#column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']
column_list = ['eig', 'et', 'inn', 'hv', 'bID']
column_list += ['pc', 'ps', 'ev', 'com', 'sbord', 'spos']

#showcolumns = ['gameID']
#showcolumns += ['visteam', 'hometeam','site', 'date', 'time', 'length']
showcolumns = ['et', 'inn', 'hv', 'bID']
showcolumns += ['pc', 'ps', 'ev', 'play', 'mdfs', 'brun',
                'com', 'sbord', 'spos']
#showcolumns += ['event_type', 'inning', 'home_or_visitor']
#showcolumns += ['batterID', 'event', 'play']
#showcolumns += ['modifiers', 'baserunning']
#showcolumns += ['comment', 'sub_batting_order', 'sub_position']

In [4]:
allevents_df = pandas.DataFrame(columns = column_list)

cnt = 0
for thisgameID,row in allgames_df.iterrows():
    if (numpy.mod(cnt,1000) == 0):
        print('Parsing game {0:,.0f}: {1}'.format(cnt, thisgameID))
    thisgame_df = pandas.DataFrame(row['events'], columns=column_list)

    thisgame_df['gameID'] = thisgameID
    
    thisgame_df = get_game_info(thisgame_df, row)

    visiting_roster_df = pandas.DataFrame(row['visiting_starters'], columns=['playerID', 'playerName', 'batting_order', 'position'])
    home_roster_df = pandas.DataFrame(row['home_starters'], columns=['playerID', 'playerName', 'batting_order', 'position'])  
    visiting_roster_df['batting_order'] = pandas.to_numeric(visiting_roster_df['batting_order'])
    visiting_roster_df['position'] = pandas.to_numeric(visiting_roster_df['position'])
    home_roster_df['batting_order'] = pandas.to_numeric(home_roster_df['batting_order'])
    home_roster_df['position'] = pandas.to_numeric(home_roster_df['position'])

    
#    print(visitor_roster)
    
#   thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
#   thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df['inn'] = pandas.to_numeric(thisgame_df['inn'])
    thisgame_df['hv'] = pandas.to_numeric(thisgame_df['hv'])

    thisgame_df = parse_events(thisgame_df)
    #thisgame

    for ing in thisgame_df['inn']:
        for side in range(0,2):
            print(thisgame_df[showcolumns][((thisgame_df['inn']) == ing) & (thisgame_df['hv'] == side)])    
            print('\n')

    allevents_df = pandas.concat((allevents_df, thisgame_df))
    
    cnt = cnt + 1
#visiting_roster_df[visiting_roster_df['position'] == 1]
#allevents_df[showcolumns]


Parsing game 0: MIN201204230
     et  inn  hv       bID  pc       ps          ev     play  mdfs brun com  \
0  play  1    0   avilm001  22  CCBBX    S7/L        S7       L               
1  play  1    0   sweer001  12  CC*BS    K           K                        
2  play  1    0   pedrd001  12  BCFX     S8/G.1-3    S8       G     1-3       
3  play  1    0   gonza003  11  CBX      7/SF/F.3-H  7        SF/F  3-H       
4  play  1    0   ortid001  22  CBF*B>B  CS2(26)     CS2(26)                  

  sbord spos  
0             
1             
2             
3             
4             


     et  inn  hv       bID  pc       ps           ev     play mdfs brun com  \
5  play  1    1   spand001  32  CBBBCFB  W            W                       
6  play  1    1   carrj001  12  CFBX     6!4(1)3/GDP  6!4(1)3  GDP            
7  play  1    1   mauej001  02  CFFX     S8/L         S8       L              
8  play  1    1   willj004  22  CCB*BS   K            K                       

  sbord 